In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
%pip install -q torch transformers datasets accelerate peft evaluate rouge_score

  DEPRECATION: Building 'rouge_score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge_score'. Discussion can be found at https://github.com/pypa/pip/issues/6334
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os

# Replace with your actual dataset folder name
dataset_path = '/kaggle/input/meddialogue'

print("Files in dataset folder:")
print(os.listdir(dataset_path))

Files in dataset folder:


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/meddialogue'

In [6]:
import json

with open("/kaggle/input/meddialogue/english-train.json", "r") as f:
    train_data = json.load(f)

with open("/kaggle/input/meddialogue/english-dev.json", "r") as f:
    val_data = json.load(f)

with open("/kaggle/input/meddialogue/english-test.json", "r") as f:
    test_data = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/meddialogue/english-train.json'

In [7]:
from datasets import Dataset

def format_dialogue_data(data):
    formatted = []
    for item in data:
        utterances = item.get("utterances", [])
        if len(utterances) >= 2 and utterances[0].lower().startswith("patient") and utterances[1].lower().startswith("doctor"):
            question = utterances[0].replace("patient:", "").strip()
            answer = utterances[1].replace("doctor:", "").strip()
            text = f"Patient: {question}\nDoctor: {answer} [Disclaimer: This is not medical advice.]"
            formatted.append({"text": text})
    return formatted

train_samples = format_dialogue_data(train_data)
val_samples = format_dialogue_data(val_data)
test_samples = format_dialogue_data(test_data)

train_dataset = Dataset.from_list(train_samples)
val_dataset = Dataset.from_list(val_samples)
test_dataset = Dataset.from_list(test_samples)

NameError: name 'train_data' is not defined

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# To resume from checkpoint, change this path
model_path = "gpt2-medium"  # or "mediguide_checkpoint_epoch2"

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

2025-05-25 03:20:12.683969: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748143212.853020      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748143212.902070      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/482 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader

tokenized_train.set_format("torch", columns=["input_ids", "attention_mask"])
train_loader = DataLoader(tokenized_train, batch_size=2, shuffle=True)


In [ ]:
from torch.optim import AdamW
from transformers import get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 2
num_training_steps = len(train_loader) * num_epochs

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)


In [ ]:
from tqdm import tqdm

model.train()

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")
    loop = tqdm(train_loader)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["input_ids"]
        )
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

    # ✅ Save after each epoch
    checkpoint_path = f"mediguide_checkpoint_epoch{epoch+1}"
    model.save_pretrained(checkpoint_path)
    tokenizer.save_pretrained(checkpoint_path)


Epoch 1


Epoch 1: 100%|██████████| 241/241 [1:09:43<00:00, 17.36s/it, loss=1]    


Epoch 2


Epoch 2: 100%|██████████| 241/241 [1:09:15<00:00, 17.24s/it, loss=0.739]


In [ ]:
# ✅ Save model and tokenizer after Epoch 2
model.save_pretrained("/kaggle/working/mediguide_checkpoint_epoch2")
tokenizer.save_pretrained("/kaggle/working/mediguide_checkpoint_epoch2")

# ✅ Zip the folder so it can be downloaded
!zip -r /kaggle/working/mediguide_checkpoint_epoch2.zip /kaggle/working/mediguide_checkpoint_epoch2


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/mediguide_checkpoint_epoch2/ (stored 0%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/vocab.json (deflated 59%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/tokenizer.json (deflated 82%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/model.safetensors (deflated 7%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/tokenizer_config.json (deflated 54%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/merges.txt (deflated 53%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/config.json (deflated 52%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/special_tokens_map.json (deflated 60%)
  adding: kaggle/working/mediguide_checkpoint_epoch2/generation_config.json (deflated 24%)


In [ ]:
from IPython.display import FileLink
FileLink(r'mediguide_checkpoint_epoch2.zip')

/kaggle/working/mediguide_checkpoint_epoch2.zip

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the saved model
model = AutoModelForCausalLM.from_pretrained("mediguide_checkpoint_epoch2")
tokenizer = AutoTokenizer.from_pretrained("mediguide_checkpoint_epoch2")
tokenizer.pad_token = tokenizer.eos_token

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
import json
from datasets import Dataset

# Load validation data
with open("/kaggle/input/meddialogue/english-dev.json", "r") as f:
    val_data = json.load(f)

# Format it into the proper prompt/response format
def format_dialogue_data(data):
    formatted = []
    for item in data:
        utterances = item.get("utterances", [])
        if len(utterances) >= 2 and utterances[0].lower().startswith("patient") and utterances[1].lower().startswith("doctor"):
            question = utterances[0].replace("patient:", "").strip()
            answer = utterances[1].replace("doctor:", "").strip()
            text = f"Patient: {question}\nDoctor:"
            formatted.append({"prompt": text, "reference": answer})
    return formatted

eval_data = format_dialogue_data(val_data)


In [ ]:
eval_data = eval_data[:100]  # Use 100 samples for ROUGE evaluation


In [ ]:
model.eval()

generated_texts = []
reference_texts = []

for sample in eval_data:
    inputs = tokenizer(sample["prompt"], return_tensors="pt", truncation=True).to(device)

    # ✅ Use attention_mask + generate only new tokens (e.g., 50 tokens after prompt)
    output_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50,  # Only generate up to 50 new tokens after the prompt
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and extract the doctor's response
    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    if "Doctor:" in generated:
        generated_answer = generated.split("Doctor:")[-1].strip()
    else:
        generated_answer = generated.strip()

    generated_texts.append(generated_answer)
    reference_texts.append(sample["reference"])


In [ ]:
from evaluate import load

rouge = load("rouge")

results = rouge.compute(predictions=generated_texts, references=reference_texts)

# Display results
print("ROUGE Evaluation Results:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")


ROUGE Evaluation Results:
rouge1: 0.1990
rouge2: 0.0536
rougeL: 0.1481
rougeLsum: 0.1480


In [ ]:
import torch
import math

model.eval()
total_loss = 0
num_batches = 0

# You can reduce batch size or sample count if this is slow
for sample in eval_data[:100]:  # Evaluate on first 100 samples
    inputs = tokenizer(
        sample["prompt"] + sample["reference"],
        return_tensors="pt",
        max_length=512,
        truncation=True,
        padding="max_length"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        total_loss += loss.item()
        num_batches += 1

# Final perplexity
avg_loss = total_loss / num_batches
perplexity = math.exp(avg_loss)

print(f"Average Loss: {avg_loss:.4f}")
print(f"Perplexity (PPL): {perplexity:.2f}")


Average Loss: 1.2206
Perplexity (PPL): 3.39
